<a href="https://colab.research.google.com/github/Isafon/ECE528/blob/main/ECE528_ASN1_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Isa Fontana ASN1 Q1
import tensorflow as tf, numpy as np, random, os
tf.random.set_seed(42); np.random.seed(42); random.seed(42)
print(tf.__version__)
!nvidia-smi

2.19.0
/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras

In [ ]:
#get data
(train_images, train_labels), (test_images, test_labels) = \
      keras.datasets.mnist.load_data()

In [ ]:
# scale model & add channel dimension
train_images = train_images.astype("float32") / 255.0
test_images  = test_images.astype("float32")  / 255.0
train_images = np.expand_dims(train_images, -1)  # (60000, 28, 28, 1)
test_images  = np.expand_dims(test_images, -1)   # (10000, 28, 28, 1)

In [ ]:
#setup model
# model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(28, 28)),
#     keras.layers.Dense(10, activation = "softmax")
# ])

from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(32, 3, padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPool2D(),
    layers.Dropout(0.25),

    layers.Conv2D(64, 3, padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPool2D(),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax"),
])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 26, 26, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 11, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 271,594 (1.04 MB)

 Trainable params: 271,402 (1.04 MB)

 Non-trainable params: 192 (768.00 B)

In [ ]:
#compile model
# model.compile(optimizer='sgd',
#           loss='sparse_categorical_crossentropy',
#           metrics=['accuracy'])
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
#train model
# model.fit(train_images, train_labels, epochs=5)
es = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=3, restore_best_weights=True
)

history = model.fit(
    train_images, train_labels,
    epochs=20,                 # early stopping will stop sooner if done
    batch_size=128,
    validation_split=0.1,
    verbose=2,
    callbacks=[es],
)

Epoch 1/20
422/422 - 196s - 465ms/step - accuracy: 0.8539 - loss: 0.4568 - val_accuracy: 0.8752 - val_loss: 0.4400
Epoch 2/20
422/422 - 179s - 424ms/step - accuracy: 0.9600 - loss: 0.1374 - val_accuracy: 0.9905 - val_loss: 0.0330
Epoch 3/20
422/422 - 202s - 479ms/step - accuracy: 0.9705 - loss: 0.1017 - val_accuracy: 0.9912 - val_loss: 0.0340
Epoch 4/20
422/422 - 207s - 490ms/step - accuracy: 0.9769 - loss: 0.0797 - val_accuracy: 0.9917 - val_loss: 0.0357
Epoch 5/20
422/422 - 206s - 489ms/step - accuracy: 0.9806 - loss: 0.0665 - val_accuracy: 0.9913 - val_loss: 0.0316
Epoch 6/20
422/422 - 188s - 445ms/step - accuracy: 0.9834 - loss: 0.0561 - val_accuracy: 0.9922 - val_loss: 0.0341
Epoch 7/20
422/422 - 189s - 447ms/step - accuracy: 0.9849 - loss: 0.0513 - val_accuracy: 0.9933 - val_loss: 0.0305
Epoch 8/20
422/422 - 198s - 469ms/step - accuracy: 0.9857 - loss: 0.0471 - val_accuracy: 0.9925 - val_loss: 0.0270
Epoch 9/20
422/422 - 177s - 419ms/step - accuracy: 0.9874 - loss: 0.0415 - val_a

In [ ]:
#evaluate

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
print("Test accuracy:", test_acc)

Test accuracy: 0.9934999942779541


In [ ]:
# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step


In [ ]:
#Raw predictions
predictions

array([[1.38293438e-22, 3.43638567e-16, 1.09557730e-12, 2.78784399e-15,
        2.46217392e-19, 1.97282689e-20, 7.12212937e-31, 1.00000000e+00,
        3.75940224e-24, 6.55652832e-14],
       [1.59887809e-13, 1.40351506e-11, 1.00000000e+00, 6.49014940e-14,
        2.07193773e-13, 3.27645388e-19, 1.61300721e-14, 5.22593155e-15,
        9.46355353e-15, 9.36661315e-16],
       [1.20405508e-11, 9.99999285e-01, 5.63964306e-07, 9.78370943e-12,
        1.01135115e-07, 1.25699984e-09, 4.32666042e-10, 3.05171746e-08,
        1.89504523e-09, 5.53801405e-09],
       [9.99990344e-01, 1.43250842e-10, 8.87887097e-10, 4.60302803e-11,
        7.99673550e-10, 2.72116374e-09, 9.45802094e-06, 5.03323383e-10,
        3.54664600e-08, 2.09738090e-07],
       [2.70984236e-16, 1.98573471e-12, 4.77555273e-10, 5.43196604e-18,
        1.00000000e+00, 3.13245787e-17, 3.14297931e-13, 1.63768365e-14,
        5.93806671e-11, 4.45946675e-08]], dtype=float32)

In [ ]:
# Print our model's predictions
print(np.argmax(predictions, axis=1))

[7 2 1 0 4]


In [ ]:
# Check our predictions against the ground truths
print(test_labels[:5]) # [7, 2, 1, 0, 4]

[7 2 1 0 4]


**Recap of what I did:**

I replaced the baseline MNIST model (Flatten -> Dense) with a small CNN that captures spatial patterns in the digits. The network uses two Conv blocks, each block is Conv2D -> BatchNorm -> Conv2D -> MaxPool -> Dropout followed by Flatten, Dense (128, ReLU), Dropout, and a 10-class softmax. I trained only on the training set (with a 10% validation split for early stopping) using Adam, then evaluated once on the test set. This improved accuracy from ~91% to 99.35% on the test set, exceeding the 99.3% target while keeping the model compact & fast.